# RAG Application

![Simple RAG](../images/simple_rag.png)

In this notebook, we're going to set up a simple RAG application that we'll be using as we learn more about LangSmith.

RAG (Retrieval Augmented Generation) is a popular technique for providing LLMs with relevant documents that will enable them to better answer questions from users.

In our case, we are going to index some LangSmith documentation!

LangSmith makes it easy to trace any LLM application, no LangChain required!

### Setup

Make sure you set your environment variables, including your OpenAI API key.

In [2]:
!pip install --quiet -U langchain-google-genai langgraph langgraph-sdk langgraph-checkpoint-sqlite langsmith langchain-community langchain-core
!pip install --quiet notebook python-dotenv lxml scikit-learn pandas pyarrow

In [3]:
!pip install --quiet python-dotenv

In [4]:
from dotenv import load_dotenv
load_dotenv(".env")

True

In [5]:
# You can set them inline!
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"

In [6]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

True

### Simple RAG application

In [14]:
from langsmith import traceable
# from openai import OpenAI
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever

import os
from google import genai

# MODEL_PROVIDER = "openai"
# MODEL_NAME = "gpt-4o-mini"
MODEL_NAME = "gemini-2.5-flash"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the latest question in the conversation.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.
"""

# openai_client = OpenAI()
client = genai.Client(api_key=os.getenv('GOOGLE_API_KEY'))
nest_asyncio.apply()
retriever = get_vector_db_retriever()

"""
retrieve_documents
- Returns documents fetched from a vectorstore based on the user's question
"""
@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

"""
generate_response
- Calls `call_gemini` to generate a model response after formatting inputs
"""
@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "user",
            "parts": [
                {"text": RAG_SYSTEM_PROMPT},
                {"text": f"Context: {formatted_docs} \n\n Question: {question}"}
            ]
        }
    ]
    return call_gemini(messages)

"""
call_gemini
- Returns the chat completion output from OpenAI
"""
@traceable(run_type="llm")
def call_gemini(
    messages: List[dict], model: str = MODEL_NAME) -> str:

    # return openai_client.chat.completions.create(
    #     model=model,
    #     messages=messages,
    #     temperature=temperature,
    # )

    return client.models.generate_content(
    model=model, contents=messages
    )

"""
langsmith_rag
- Calls `retrieve_documents` to fetch documents
- Calls `generate_response` to generate a response based on the fetched documents
- Returns the model response
"""
@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    # Access the text content from the response
    return response.candidates[0].content.parts[0].text

Loading existing vector store from: /tmp/union_local.parquet


This should take a little less than a minute. We are indexing and storing LangSmith documentation in a SKLearn vector database.

In [15]:
question = "What is LangSmith used for?"
ai_answer = langsmith_rag(question)
print(ai_answer)

LangSmith is used for storing and processing trace data and feedback. It utilizes Clickhouse as its primary storage engine for this purpose. LangSmith can be run via Kubernetes or Docker in a cloud environment.


### Let's take a look in LangSmith!